# Assuming that strain is know (ideally 0 as in ruby standard)

In [9]:
import sys, os

import numpy as np
from scipy import optimize

import cPickle

from scipy.stats import chisquare

import ipywidgets as widgets
from IPython.display import display

import yaml
from scipy.linalg.matfuncs import logm
import scipy.optimize as opt

from hexrd import config
from hexrd import matrixutil as mutil

from hexrd.coreutil import initialize_experiment

from hexrd.fitgrains import get_instrument_parameters

from hexrd.xrd import distortion as dFuncs
from hexrd.xrd import fitting
from hexrd.xrd import material
from hexrd.xrd import transforms as xf
from hexrd.xrd import transforms_CAPI as xfcapi
from hexrd.xrd.crystallography import processWavelength
from hexrd.xrd.distortion import GE_41RT as distortion_func

# parameters
kev_conv_fac = processWavelength(1.)
bVec_ref = xf.bVec_ref
eta_ref = xf.eta_ref
vInv_ref = xf.vInv_ref
zvec3 = np.zeros(3)

# User input parameters

In [10]:
cfg_filename = 'config.yml'
scan_id = 1
grain_id = 3

In [11]:
"""
####### BASE CONFIG
"""
# read config
cfg = config.open(cfg_filename)[scan_id]

# crystallography
pd, reader, detector = initialize_experiment(cfg)
bMat = np.ascontiguousarray(pd.latVecOps['B'])
wlen0 = pd.wavelength * 1. # Angstroms
en0 = kev_conv_fac/wlen0 # keV

# directories and names
working_dir = cfg.working_dir
analysis_name = cfg.analysis_name
analysis_dir = cfg.analysis_dir
parfilename = cfg.instrument.parameters
spots_filename = os.path.join(analysis_dir, 'spots_%05d.out' % grain_id)

print "INFO: Analysis name: '%s'" %analysis_name

"""
####### INSTRUMENT
"""
# config
instr_cfg = get_instrument_parameters(cfg)

# transform
tilt_angles = instr_cfg['detector']['transform']['tilt_angles']
rMat_d = xf.makeDetectorRotMat(tilt_angles) 
tVec_d = instr_cfg['detector']['transform']['t_vec_d']

# oscillation stage
chi = instr_cfg['oscillation_stage']['chi']
tVec_s = instr_cfg['oscillation_stage']['t_vec_s']

# distortion
distortion_params = instr_cfg['detector']['distortion']['parameters']
distortion = (distortion_func, distortion_params)

# intensity
int_cutoff = instr_cfg['detector']['saturation_level']

"""
####### GRAIN
"""
# load grain parameters
grain_params = np.loadtxt(os.path.join(analysis_dir, 'grains.out'), ndmin=2)[grain_id, :]
expMap_c = grain_params[3:6]
tVec_c = grain_params[6:9]
#vInv_s = grain_params[9:15]
vInv_s = np.r_[1., 1., 1., 0., 0., 0.] # reset strains

# omega period 
omePeriod = np.radians(cfg.find_orientations.omega.period)

# load reflections
refl_table = np.loadtxt(spots_filename) # load pull_spots output table
valid_reflections = refl_table[:, 0] >= 0
not_saturated = refl_table[:, 6] < int_cutoff
print "INFO: %d of %d reflections are valid" %(sum(valid_reflections), len(refl_table))
print "INFO: %d of %d valid reflections be are below saturation threshold of %d" \
    %(sum(not_saturated), sum(valid_reflections), int_cutoff)

idx = np.logical_and(valid_reflections, not_saturated) # select valid reflections

# strip fields
hkls = refl_table[idx, 2:5].T # must be column vectors
xyo_det = refl_table[idx, -3:] # these are the cartesian centroids + ome
xyo_det[:, 2] = xf.mapAngle(xyo_det[:, 2], omePeriod)

nrefl = sum(idx)

# parameter list
detector_params = fitting.geomParamsToInput(tilt_angles, chi, expMap_c, tVec_d, tVec_s, tVec_c, distortion_params)

INFO: Analysis name: 'scan_17'
INFO: 1365 of 1442 reflections are valid
INFO: 1339 of 1365 valid reflections be are below saturation threshold of 14000


In [12]:
def fit_wavelength(wlen):
    """
    objective function harness for fitting lambda base on grain fits
    """
    g_initial = np.hstack([expMap_c.flatten(), tVec_c.flatten(), vInv_s.flatten()])
    gFlag = np.array([np.ones(6), np.zeros(6)], dtype=bool).flatten()

    # fit grain with current wavelength
    return fitting.objFuncFitGrain(g_initial[gFlag], g_initial, gFlag, 
                                   detector_params, xyo_det, 
                                   hkls, bMat, wlen, 
                                   bVec_ref, eta_ref, 
                                   distortion_func, distortion_params,
                                   omePeriod, simOnly=False)

# least squares for wavelength
results = optimize.leastsq(fit_wavelength, wlen0)
wlen_fit = results[0][0]

print "Initial wavelength (energy): %.6f (%.5f)\nOptimized wavelength (energy): %.6f (%.5f)" \
    %(wlen0, kev_conv_fac/wlen0, wlen_fit, kev_conv_fac/wlen_fit)

Initial wavelength (energy): 0.202200 (61.31768)
Optimized wavelength (energy): 0.202332 (61.27745)
